# user based collaborative filtering

https://medium.com/@srinidhi14vaddy/collaborative-filtering-movie-recommendation-60461c7ef897

https://www.kaggle.com/srinidhi14vaddy/collaborative-filtering-movie-recommendation/notebook

It uses other users to recommend items to the input user. It attempts to find users that have similar preferences and opinions as the input and then recommends items that they have liked to the input. There are several methods of finding similar users (Even some making use of Machine Learning), and the one we will be using here is going to be based on the Pearson Correlation Function.

In [2]:
# Import modules
import numpy as np
import pandas as pd
from math import sqrt
import matplotlib.pyplot as plt
%matplotlib inline

## Import datasets

In [3]:
rating = pd.read_csv('data/train.csv')
movie = pd.read_csv('data/movies.csv')

Quick view of the two dataframes, the rows and the columns

In [4]:
rating.head()

,userId,movieId,rating,timestamp
0,5163,57669,4.0,1518349992
1,106343,5,4.5,1206238739
2,146790,5459,5.0,1076215539
3,106362,32296,2.0,1423042565
4,9041,366,3.0,833375837


In [5]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


### movie df
Using pandas' replace function, we remove the year from the title in the movie dataframe and add a separate year column. This is done using regular expressions. Find a year stored between parentheses. We specify the parantheses so we don't conflict with movies that have years in their titles.



First we create the new Year column in the movie dataframe.

In [6]:
movie['year'] = movie.title.str.extract('(\(\d\d\d\d\))',expand=False)

#Removing the parentheses
movie['year'] = movie.year.str.extract('(\d\d\d\d)',expand=False)

Now removing the year from the title in the Title column.

In [8]:
#Removing the years from the 'title' column
movie['title'] = movie.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movie['title'] = movie['title'].apply(lambda x: x.strip())

<ipython-input-8-9fa5ec22f204>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  movie['title'] = movie.title.str.replace('(\(\d\d\d\d\))', '')


Collaborative filtering doesn't recommend based on the features of the movie. The recommendation is based on the likes and dislikes or ratings of the neighbours or other users. So we will drop the genre column, since there is no use of it.

In [10]:
movie.drop(columns=['genres'], inplace=True)

### Rating df
Now, coming to the ratings dataframe, we have the movieId column that is common with the movie dataframe. Each user has given multiple ratings for different movies. The column Timestamp is not needed for the recommendation system. So we can drop it.

In [12]:
rating.drop(columns=['timestamp'],inplace=True)

The technique is called Collaborative Filtering, which is also known as User-User Filtering. As hinted by its alternate name, this technique uses other users to recommend items to the input user. It attempts to find users that have similar preferences and opinions as the input and then recommends items that they have liked to the input. There are several methods of finding similar users (Even some making use of Machine Learning), and the one we will be using here is going to be based on the Pearson Correlation Function.

The process for creating a User Based recommendation system is as follows:

- Select a user with the movies the user has watched
- Based on his rating to movies, find the top X neighbours
- Get the watched movie record of the user for each neighbour.
- Calculate a similarity score using some formula
- Recommend the items with the highest score

In [35]:
# Identify five movies the user has watched
user = [
            #{'title':'Breakfast Club, The', 'rating':4},
            {'title':'Toy Story', 'rating':2.5},
            #{'title':'Jumanji', 'rating':3},
            #{'title':"Pulp Fiction", 'rating':4.5},
            #{'title':'Akira', 'rating':5}
         ] 
inputMovie = pd.DataFrame(user)
inputMovie

,title,rating
0,Toy Story,2.5


We need to now add the movieId column from the movie dataframe into the inputMovie Dataframe. First filter out the rows that contain the input movies' title and then merging this subset with the input dataframe. We also drop unnecessary columns for the input to save memory space.

In [36]:
#Filtering out the movies by title
Id = movie[movie['title'].isin(inputMovie['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovie = pd.merge(Id, inputMovie)
#Dropping information we won't use from the input dataframe
inputMovie = inputMovie.drop('year', 1)
inputMovie

,movieId,title,rating
0,1,Toy Story,2.5


Finding the users who have seen the same movies from the rating dataframe With the movie ID's in our input, we can now get the subset of users that have watched and reviewed the movies in our input.

In [37]:
#Filtering out users that have watched movies that the input has watched and storing it
users = rating[rating['movieId'].isin(inputMovie['movieId'].tolist())]
users.head()

,userId,movieId,rating
119,158849,1,5.0
154,97203,1,5.0
194,161871,1,3.0
720,45117,1,4.0
1305,27431,1,5.0


In [38]:
users.shape

(23062, 3)

In [39]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = users.groupby(['userId'])

In [40]:
#showing one such group example by getting all the users of a particular uderId
userSubsetGroup.get_group(121814)

KeyError: 121814

In [41]:
#Sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

In [42]:
userSubsetGroup[0:3]

[(5,
           userId  movieId  rating
  2212756       5        1     4.0),
 (10,
           userId  movieId  rating
  3567438      10        1     3.5),
 (13,
           userId  movieId  rating
  7830044      13        1     4.0)]

Similarity of users to input user Next, we are going to compare all users to our specified user and find the one that is most similar. we're going to find out how similar each user is to the input through the Pearson Correlation Coefficient. It is used to measure the strength of a linear association between two variables. The formula for finding this coefficient between sets X and Y with N values can be seen in the image below.

The values given by the formula vary from r = -1 to r = 1, where 1 forms a direct correlation between the two entities (it means a perfect positive correlation) and -1 forms a perfect negative correlation.

In our case, a 1 means that the two users have similar tastes while a -1 means the opposite.

In [43]:
userSubsetGroup = userSubsetGroup[0:100]

In [44]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovie = inputMovie.sort_values(by='movieId')
    #Get the N for the formula
    n = len(group)
    #Get the review scores for the movies that they both have in common
    temp = inputMovie[inputMovie['movieId'].isin(group['movieId'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp['rating'].tolist()
    #put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(n)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(n)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(n)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorDict[name] = 0   

In [26]:
pearsonCorDict.items()

dict_items([(43983, 0.7724016533656629), (47001, -0.41860465116279033), (48197, -0.03698633607276333), (54125, 0.7817876057249286), (57946, 0.6168870908644989), (76334, 0.5402893024772932), (96495, 0.3083606193532053), (117801, 0.2326252139407966), (120443, 0.10783277320344019), (120901, 0.5379643898572857), (125551, 0.24041118447297743), (137196, -0.5545259368002142), (138487, 0.9644856443408245), (150141, -0.5455484570732912), (159612, 0.1903297204970176), (265, -0.50709255283711), (321, 0.7627700713964739), (3394, 0.32071349029490925), (3777, -0.8841263423928601), (4006, -0.24253562503633297), (4950, -0.10690449676496974), (5019, 0.41169348479630913), (6118, -0.6673083711820307), (7540, 0.9741763740941048), (8150, 0.9036961141150639), (8641, -0.435285750066007), (10106, 0.9683296637314885), (10357, -0.36140316116210053), (10496, 0.9534625892455924), (10508, 0.7071067811865475), (10717, -0.32605565837568895), (11491, -0.6454972243679028), (11778, 0.8783100656536799), (11952, 0.1), (1

In [45]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0,5
1,0,10
2,0,13
3,0,26
4,0,43


In [46]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
0,0,5
63,0,516
73,0,573
72,0,567
71,0,561


Rating of selected users to all movies We're going to do this by taking the weighted average of the ratings of the movies using the Pearson Correlation as the weight. But to do this, we first need to get the movies watched by the users in our pearsonDF from the ratings dataframe and then store their correlation in a new column called _similarityIndex". This is achieved below by merging of these two tables.

In [47]:
topUsersRating=topUsers.merge(rating, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,movieId,rating
0,0,5,216,4.0
1,0,5,47,5.0
2,0,5,219,3.0
3,0,5,432,3.0
4,0,5,784,2.0


multiply the movie rating by its weight (The similarity index), then sum up the new ratings and divide it by the sum of the weights.

We can easily do this by simply multiplying two columns, then grouping up the dataframe by movieId and then dividing two columns:

In [48]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,0,5,216,4.0,0.0
1,0,5,47,5.0,0.0
2,0,5,219,3.0,0.0
3,0,5,432,3.0,0.0
4,0,5,784,2.0,0.0


For each distinct movie, we then calculate a weighted average similarity index of all users and weighted average rating, which is then used for the weighted recommendation score for each movie.

In [49]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,0,0.0
2,0,0.0
3,0,0.0
5,0,0.0
6,0,0.0


In [50]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,NaN,1
2,NaN,2
3,NaN,3
5,NaN,5
6,NaN,6


When sorted in descending order, we get the top 10 movies with the highest recommendation score. These are the top 10 recommendations for the input user based on what others are watching!

In [51]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
1,NaN,1
2,NaN,2
3,NaN,3
5,NaN,5
6,NaN,6
7,NaN,7
8,NaN,8
9,NaN,9
10,NaN,10


In [52]:
movie.loc[movie['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
4,5,Father of the Bride Part II,1995
5,6,Heat,1995
6,7,Sabrina,1995
7,8,Tom and Huck,1995
8,9,Sudden Death,1995
9,10,GoldenEye,1995
10,11,"American President, The",1995


In [13]:
rating.head()

,userId,movieId,rating
0,5163,57669,4.0
1,106343,5,4.5
2,146790,5459,5.0
3,106362,32296,2.0
4,9041,366,3.0
